In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from datetime import datetime, timedelta
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=False)
pd.set_option('display.max_rows', 200)

Using TensorFlow backend.


In [2]:
def read_data(features, dataset_name):
    x1 = pd.read_excel(dataset_name, sheet_name=features[1])
    x2 = pd.read_excel(dataset_name, sheet_name=features[2])
    x3 = pd.read_excel(dataset_name, sheet_name=features[3]) 

    x1.date = x1.date.astype('datetime64')
    x2.date = x2.date.astype('datetime64')
    x3.date = x3.date.astype('datetime64')
    return x1, x2, x3

In [3]:
def get_range(x1, x2, x3):
    global start, end
    start = max(x1.date.min(), x2.date.min(), x3.date.min())
    end = min(x1.date.max(), x2.date.max(), x3.date.max())
    date_range = pd.date_range(start, end, freq='MS')
    return date_range

In [4]:
def get_ready_df():
    global x1, x2, x3, features
    data = pd.DataFrame(reversed(np.column_stack([x1.exchange.values, x2.inflation.values, x3.interest.values])), columns=features.values(), index=date_range)
    return data

In [153]:
def get_ready_data(date_range, to_train):
    global look_back, scaled, train_dates, test_dates
    data = get_ready_df()
    to_drop = data.shape[0] % batch_size
    if to_drop > 0:
        print('last {} instance/s dropped'.format(to_drop))
        data = data[:-1 * to_drop]
        date_range = date_range[: -1 * to_drop]
        
    scale_data(data)
    scaled_train_x = scaled[:to_train]
    scaled_train_y = scaled_train_x[:,0].reshape(-1,1)
    scaled_test_x = scaled[to_train-look_back:]
    scaled_test_y = scaled_test_x[:,0].reshape(-1,1)
    train_dates = date_range[:to_train][look_back:]
    test_dates = date_range[to_train:]
    return data, date_range, scaled_train_x, scaled_train_y, scaled_test_x, scaled_test_y

In [34]:
def scale_data(data):
    global x1_scaler, x2_scaler, x3_scaler, scaled_x1, scaled_x2, scaled_x3, scaled
    x1_scaler = MinMaxScaler()
    x2_scaler = MinMaxScaler()
    x3_scaler = MinMaxScaler()
    scaled_x1 = x1_scaler.fit_transform(data.iloc[:,0].values.reshape(-1,1))
    scaled_x2 = x2_scaler.fit_transform(data.iloc[:,1].values.reshape(-1,1))
    scaled_x3 = x3_scaler.fit_transform(data.iloc[:,2].values.reshape(-1,1))
    scaled    = np.column_stack([scaled_x1,scaled_x2,scaled_x3])

In [31]:
dataset_name="EUR_USD.xlsx"
features = {1:'exchange', 2:'inflation', 3:'interest'}
x1, x2, x3 = read_data(features, dataset_name)
date_range = get_range(x1, x2, x3)

In [33]:
x1.set_index('date', inplace=True)
x1.sort_index(ascending=False, inplace=True)
x1['year'] = x1.index.year
x1['month'] = x1.index.month
x1 = x1.loc[[ d for d in x1.index if (d in date_range) | (d - timedelta(days=1) in date_range) | (d - timedelta(days=2) in date_range) ]]
x1.drop_duplicates(['year', 'month'], keep='last', inplace=True)

x2.date = x2.date + timedelta(days=1)
x2.set_index('date', inplace=True)
x2.sort_index(ascending=False, inplace=True)
x2 = x2.loc[[ d for d in x2.index if (d in date_range)]]

x3 = x3[(x3.date >= start) & (x3.date <= end)]
x3.set_index('date', inplace=True)
x3.sort_index(ascending=False, inplace=True)
x3['year'] = x3.index.year
x3['month'] = x3.index.month
x3.drop_duplicates(['year', 'month'], keep='last', inplace=True)

In [91]:
trace1 = go.Scatter(
    x = date_range,
    y = scaled[:,0],
    mode = 'lines',
    name = 'Exchange'
)
trace2 = go.Scatter(
    x = date_range,
    y = scaled[:,1],
    mode = 'lines',
    name = 'Inflation'
)
trace3 = go.Scatter(
    x = date_range,
    y = scaled[:,2],
    mode = 'lines',
    name = 'Interest'
)
layout = go.Layout(
    title = "Min-Max Scaled EUR/USD Exchange Rates" ,
    xaxis = {'title' : "Date (months)"},
    yaxis = {'title' : "Rates"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [154]:
num_epochs = 200
look_back = 6
batch_size = 6
to_train = 156

In [155]:
eur_usd, horizon, train_x, train_y, test_x, test_y = get_ready_data(date_range, to_train)

last 1 instance/s dropped


In [156]:
train_generator = TimeseriesGenerator(train_x, train_y, length=look_back, batch_size=batch_size)
test_generator  = TimeseriesGenerator(test_x, test_y, length=look_back, batch_size=batch_size) 

In [139]:
model = Sequential()
model.add(LSTM(40, input_shape=(look_back, scaled.shape[1]), batch_size=batch_size))
model.add(Dropout(0.25))
model.add(LeakyReLU(alpha=0.5))
model.add(Dense(1))
model.compile(tf.optimizers.Adam(), loss='mse')

In [140]:
model.fit_generator(train_generator, epochs=num_epochs, verbose=1, validation_data=test_generator)

Epoch 1/200
25/25 [==============================] - 1s 26ms/step - loss: 0.1702 - val_loss: 0.0016
Epoch 2/200
25/25 [==============================] - 0s 7ms/step - loss: 0.0311 - val_loss: 0.0086
Epoch 3/200
25/25 [==============================] - 0s 7ms/step - loss: 0.0230 - val_loss: 0.0017
Epoch 4/200
25/25 [==============================] - 0s 7ms/step - loss: 0.0280 - val_loss: 0.0029
Epoch 5/200
25/25 [==============================] - 0s 7ms/step - loss: 0.0254 - val_loss: 0.0014
Epoch 6/200
25/25 [==============================] - 0s 6ms/step - loss: 0.0214 - val_loss: 5.3845e-04
Epoch 7/200
25/25 [==============================] - 0s 10ms/step - loss: 0.0181 - val_loss: 0.0016
Epoch 8/200
25/25 [==============================] - 0s 7ms/step - loss: 0.0211 - val_loss: 4.9866e-04
Epoch 9/200
25/25 [==============================] - 0s 7ms/step - loss: 0.0166 - val_loss: 3.9604e-04
Epoch 10/200
25/25 [==============================] - 0s 9ms/step - loss: 0.0186 - val_loss: 7.

25/25 [==============================] - 0s 12ms/step - loss: 0.0084 - val_loss: 3.1936e-04
Epoch 81/200
25/25 [==============================] - 0s 10ms/step - loss: 0.0095 - val_loss: 2.3750e-04
Epoch 82/200
25/25 [==============================] - 0s 9ms/step - loss: 0.0092 - val_loss: 2.1296e-04
Epoch 83/200
25/25 [==============================] - 0s 10ms/step - loss: 0.0085 - val_loss: 3.2519e-04
Epoch 84/200
25/25 [==============================] - 0s 11ms/step - loss: 0.0082 - val_loss: 2.4438e-04
Epoch 85/200
25/25 [==============================] - 0s 9ms/step - loss: 0.0088 - val_loss: 8.5214e-04
Epoch 86/200
25/25 [==============================] - 0s 11ms/step - loss: 0.0083 - val_loss: 3.4834e-04
Epoch 87/200
25/25 [==============================] - 0s 11ms/step - loss: 0.0088 - val_loss: 3.3822e-04
Epoch 88/200
25/25 [==============================] - 0s 10ms/step - loss: 0.0089 - val_loss: 4.6123e-04
Epoch 89/200
25/25 [==============================] - 0s 9ms/step - lo

Epoch 158/200
25/25 [==============================] - 0s 11ms/step - loss: 0.0061 - val_loss: 3.8922e-04
Epoch 159/200
25/25 [==============================] - 0s 9ms/step - loss: 0.0071 - val_loss: 5.9636e-04
Epoch 160/200
25/25 [==============================] - 0s 9ms/step - loss: 0.0065 - val_loss: 3.4358e-04
Epoch 161/200
25/25 [==============================] - 0s 10ms/step - loss: 0.0067 - val_loss: 2.7762e-04
Epoch 162/200
25/25 [==============================] - 0s 11ms/step - loss: 0.0073 - val_loss: 0.0014
Epoch 163/200
25/25 [==============================] - 0s 11ms/step - loss: 0.0064 - val_loss: 4.1214e-04
Epoch 164/200
25/25 [==============================] - 0s 10ms/step - loss: 0.0067 - val_loss: 0.0011
Epoch 165/200
25/25 [==============================] - 0s 11ms/step - loss: 0.0060 - val_loss: 8.3267e-04
Epoch 166/200
25/25 [==============================] - 0s 16ms/step - loss: 0.0068 - val_loss: 6.5463e-04
Epoch 167/200
25/25 [==============================] - 0

In [141]:
train_pred = model.predict_generator(train_generator)
train_pred = x1_scaler.inverse_transform(train_pred).flatten()

In [142]:
test_pred = model.predict_generator(test_generator)
test_pred = x1_scaler.inverse_transform(test_pred).flatten()

In [231]:
trace1 = go.Scatter(
    x = horizon,
    y = eur_usd.exchange,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = train_dates - timedelta(days=31),
    y = train_pred,
    mode = 'lines',
    name = 'Train'
)
trace3 = go.Scatter(
    x = test_dates - timedelta(days=31),
    y = test_pred,
    mode = 'lines',
    name = 'Test'
)
layout = go.Layout(
    title = "Time-Series Prediction of EUR/USD by LSTM" ,
    xaxis = {'title' : "Date (months)"},
    yaxis = {'title' : "Exchange Rate"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [209]:
def get_svr_data():
    global train_x, train_y, test_x, test_y
    train_x_svr = train_x[:-look_back]
    train_y_svr = train_y[look_back:].ravel()
    test_x_svr = test_x[:-look_back]
    test_y_svr = test_y[look_back:].ravel()
    return train_x_svr, train_y_svr, test_x_svr, test_y_svr

In [210]:
train_x_svr, train_y_svr, test_x_svr, test_y_svr = get_svr_data()

In [226]:
svr_rbf = SVR(kernel='rbf', C=1000, gamma=0.5)
svr_rbf.fit(train_x_svr, train_y_svr)

SVR(C=1000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.5,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [227]:
svr_rbf.score(train_x_svr, train_y_svr)

0.717100165559406

In [228]:
svr_rbf.score(test_x_svr, test_y_svr)

-1.8441436322015674

In [229]:
train_pred_svr = svr_rbf.predict(train_x_svr)
train_pred_svr = x1_scaler.inverse_transform(train_pred_svr.reshape(-1,1)).flatten()
test_pred_svr = svr_rbf.predict(test_x_svr)
test_pred_svr = x1_scaler.inverse_transform(test_pred_svr.reshape(-1,1)).flatten()

In [230]:
trace1 = go.Scatter(
    x = date_range,
    y = eur_usd.exchange,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = train_dates,
    y = train_pred_svr,
    mode = 'lines',
    name = 'Train'
)
trace3 = go.Scatter(
    x = test_dates,
    y = test_pred_svr,
    mode = 'lines',
    name = 'Test'
)
layout = go.Layout(
    title = "Time-Series Prediction of EUR/USD by SVR" ,
    xaxis = {'title' : "Date (months)"},
    yaxis = {'title' : "Exchange Rate"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()